# show reconstruct image

In [1]:
import pickle
import yaml
import os
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn.functional as F

In [2]:
########################################################################
# load config
########################################################################
with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [3]:
########################################################################
# Setting I/O path
########################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
eval_path = INPUT_ROOT + "/eval_test"
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
RESULT_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/result'
RECONS_OUTDIR = OUTPUT_ROOT +'/eval_reconstruct_img'
PKL_DIR = OUTPUT_ROOT +'/pkl'
os.makedirs(MODEL_DIR, exist_ok=True)

# eval

## import library

In [4]:
########################################################################
# import default python-library
########################################################################
import os
import glob
import csv
import re
import itertools
import sys
from collections import defaultdict
########################################################################


########################################################################
# import additional python-library
########################################################################
import numpy
from sklearn import metrics
import common as com
import pytorch_modeler as modeler
from pytorch_model import DAGMM as Model
import torch.utils.data
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
########################################################################
import eval_functions as eval_func
from pytorch_utils import to_var

## Setting

In [5]:
########################################################################
# load config
########################################################################
with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
########################################################################
# Setting seed
########################################################################
modeler.set_seed(42)

In [7]:
########################################################################
# Setting I/O path
########################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
eval_path = INPUT_ROOT + "/eval_test"
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
RESULT_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/result'
RECONS_OUTDIR = OUTPUT_ROOT +'/eval_reconstruct_img'
os.makedirs(MODEL_DIR, exist_ok=True)

In [8]:
########################################################################
# for original function
########################################################################
param = {}
param["dev_directory"] = dev_path
param["eval_directory"] = eval_path
param["model_directory"] = MODEL_DIR
param["result_directory"] = RESULT_DIR
param["result_file"] = 'result.csv'

from sklearn.metrics import mean_squared_error as mse

def calc_time_anomaly(x, y, label, file_name):
    fig = plt.figure(figsize=(10,5)) # width, height
    fig.suptitle('label={}'.format(int(label)))
    time_anomaly = np.zeros((x.shape[0]))
    for frame in range(x.shape[0]):
        time_anomaly[frame] = mse(y[frame,:],x[frame,:])
    plt.plot(time_anomaly)
    plt.title(f'label:{label}')

time_anomaly.mean()

plt.plot(time_anomaly)

plt.figure(figsize=(5,5))
plt.imshow(np.abs(x-y), aspect='auto')

## run eval

In [9]:
def make_reconstruct_img(x, y, label, file_name):
    fig = plt.figure(figsize=(10,5)) # width, height
    fig.suptitle('label={}'.format(int(label)))
    ax1 = fig.add_subplot(121, title='x') # 明示的にAxesを作成する
    sns.heatmap(x.T, ax=ax1) # ax1を参照するようにする
    ax2 = fig.add_subplot(122, title='y')
    sns.heatmap(y.T, ax=ax2)
    fig.savefig('{}.png'.format(file_name))

In [10]:
mode = 'dev'

In [11]:
from torchvision import transforms
import preprocessing as prep

class extract_waveform(object):
    """
    wavデータロード(波形)
    
    Attributes
    ----------
    sound_data : waveform
    """
    def __init__(self, sound_data=None):
        self.sound_data = sound_data
    
    def __call__(self, sample):
        self.sound_data = com.file_load(sample['wav_name'],
                                        sr=config['preprocessing']['sample_rate'],
                                        mono=config['preprocessing']['mono'])
        self.sound_data = self.sound_data[0]
        self.label = np.array(sample['label'])
        self.wav_name = sample['wav_name']
        
        return {'feature': self.sound_data, 'label': self.label, 'wav_name': self.wav_name}

class ToTensor(object):
    """
    Convert ndarrays in sample to Tensors.
    """

    def __call__(self, sample):
        feature, label, wav_name = sample['feature'], sample['label'], sample['wav_name']
        
        return {'feature': torch.from_numpy(feature).float(), 'label': torch.from_numpy(label), 'wav_name': wav_name}

class DCASE_task2_Dataset_test(torch.utils.data.Dataset):
    '''
    Attribute
    ----------
    
    '''
    
    def __init__(self, file_list, transform=None):
        self.transform = transform
        self.file_list = file_list
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        file_path = self.file_list[idx]
        # ファイル名でlabelを判断
        if "normal" in file_path:
            label = 0
        else:
            label = 1
        
        sample = {'wav_name':file_path, 'label':np.array(label)}
        sample = self.transform(sample)
        
        return sample

def make_dataloader_train(paths):
    transform = transforms.Compose([
        extract_waveform(),
        ToTensor()
    ])
    dataset = DCASE_task2_Dataset_test(paths, transform=transform)
    
    train_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=config['fit']['batch_size'],
        shuffle=True,
        num_workers=2,
        pin_memory=True
        )
    
    return test_loader
    
def make_dataloader_test(paths):
    transform = transforms.Compose([
        extract_waveform(),
        ToTensor()
    ])
    dataset = DCASE_task2_Dataset_test(paths, transform=transform)
    
    test_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=config['fit']['batch_size'],
        shuffle=False,
        num_workers=2,
        pin_memory=True
        )
    
    return test_loader

In [12]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths = sorted(dev_train_paths)
    # add_dev train
    add_train_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_paths = sorted(add_train_paths)
    # valid
    dev_valid_paths = ["{}/{}/test/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    dev_valid_paths = sorted(dev_valid_paths)
    
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths + add_train_paths
    train_paths[machine_type]['valid'] = dev_valid_paths

In [13]:
#def run_eval(param, mode):
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# make output result directory
os.makedirs(RESULT_DIR, exist_ok=True)

# load base directory
dirs = com.select_dirs(param=param, mode=mode)

# initialize lines in csv for AUC and pAUC
csv_lines = []


# loop of the base directory
for idx, target_dir in enumerate(dirs):
    com.logger.info("===========================")
    com.logger.info("[{idx}/{total}] {dirname}".format(
        dirname=target_dir, idx=idx+1, total=len(dirs)))

    machine_type = os.path.split(target_dir)[1]

    com.logger.info("============== MODEL LOAD ==============")

    model_file = "{model}/{machine_type}_model.pth".format(
        model=param["model_directory"],
        machine_type=machine_type)

    if not os.path.exists(model_file):
        com.logger.error("{} model not found ".format(machine_type))
        sys.exit(-1)

    # define AE model
    model = Model(sample_rate=config['preprocessing']['sample_rate'],
                  window_size=config['preprocessing']['window_size'],
                  hop_size=config['preprocessing']['hop_size'],
                  mel_bins=config['preprocessing']['mel_bins'],
                  fmin=config['preprocessing']['fmin'],
                  fmax=config['preprocessing']['fmax'],
                  latent_size=config['fit']['latent_size'],
                  mixture_size=config['fit']['mixture_size']).to(device)
    model.eval()
    model.load_state_dict(torch.load(model_file))

    if mode:
        # results by type
        csv_lines.append([machine_type])
        csv_lines.append(["id", "AUC", "pAUC"])
        performance = []

    machine_id_list = eval_func.get_machine_id_list_for_test(target_dir)
    recons_outpath = RECONS_OUTDIR + '/' + machine_type
    os.makedirs(recons_outpath, exist_ok=True)
    
    # calc train GMM param
    com.logger.info(f"============== CALC GMM PARAM : {machine_type} ==============")
    
    train_loader = make_dataloader_test(train_paths[machine_type]['train'])
    N = 0
    mu_sum = 0
    cov_sum = 0
    gamma_sum = 0
    with torch.no_grad():
        for sample in tqdm(train_loader):
            input_data = sample['feature']
            input_data = to_var(input_data)
            nn_out = model(input_data, device)
            tr_phi, tr_mu, tr_cov = model.compute_gmm_params(nn_out['z'], nn_out['gamma'])
            tr_phi, tr_mu, tr_cov = tr_phi.to('cpu'), tr_mu.to('cpu'), tr_cov.to('cpu')
            batch_gamma_sum = torch.sum(nn_out['gamma'].to('cpu'), dim=0).to('cpu')
            gamma_sum += batch_gamma_sum
            mu_sum += tr_mu * batch_gamma_sum.unsqueeze(-1) # keep sums of the numerator only
            cov_sum += tr_cov * batch_gamma_sum.unsqueeze(-1).unsqueeze(-1) # keep sums of the numerator only
            N += input_data.size(0)
    train_phi = gamma_sum / N
    train_mu = mu_sum / gamma_sum.unsqueeze(-1)
    train_cov = cov_sum / gamma_sum.unsqueeze(-1).unsqueeze(-1)
    print("N:",N)
    print("phi :\n",train_phi)
    print("mu :\n",train_mu)
    print("cov :\n",train_cov)
    
    # evaluation
    for id_str in machine_id_list:

        # load list of test files
        test_files, y_true = eval_func.test_file_list_generator(target_dir, id_str, mode)

        # setup anomaly score file path
        anomaly_score_csv = \
            "{result}/anomaly_score_{machine_type}_{id_str}.csv"\
            .format(result=param["result_directory"],
                    machine_type=machine_type,
                    id_str=id_str)
        anomaly_score_list = []

        com.logger.info(
            "============== BEGIN TEST FOR A MACHINE ID ==============")

        y_pred = []
        anomaly_count = 0
        normal_count = 0
        
        test_loader = make_dataloader_test(test_files)
        start_idx = 0
        end_idx = 0
        slicing = None
        with torch.no_grad():
            for it, data in enumerate(tqdm(test_loader)):
                try:
                    feature = data['feature']
                    feature = to_var(feature)
                    label = data['label']
                    file_path = data['wav_name']
                    # reconstruction through auto encoder in pytorch
                    with torch.no_grad():
                        nn_out = model(feature)
                        z, _ = nn_out['z'], nn_out['gamma']
                        sample_energy, cov_diag = model.compute_energy(z, phi=to_var(train_phi), mu=to_var(train_mu), cov=to_var(train_cov), size_average=False)
                        preds = sample_energy.data.cpu().numpy()
                        if it == 0:
                            y_pred = preds.copy()
                        else:
                            y_pred = np.concatenate([y_pred, preds], axis=0)

                    for idx in range(len(file_path)):
                        anomaly_score_list.append([os.path.basename(file_path[idx]), preds[idx]])
                except FileNotFoundError:
                    com.logger.error("file broken!!")

        # save anomaly score
        eval_func.save_csv(save_file_path=anomaly_score_csv,
                           save_data=anomaly_score_list)
        com.logger.info(
            "anomaly score result ->  {}".format(anomaly_score_csv))

        if mode:
            # append AUC and pAUC to lists
            auc = metrics.roc_auc_score(y_true, y_pred)
            p_auc = metrics.roc_auc_score(
                y_true, y_pred, max_fpr=config["etc"]["max_fpr"])
            csv_lines.append([id_str.split("_", 1)[1], auc, p_auc])
            performance.append([auc, p_auc])
            com.logger.info("AUC : {}".format(auc))
            com.logger.info("pAUC : {}".format(p_auc))

        com.logger.info(
            "============ END OF TEST FOR A MACHINE ID ============")

    if mode:
        # calculate averages for AUCs and pAUCs
        averaged_performance = numpy.mean(
            numpy.array(performance, dtype=float), axis=0)
        csv_lines.append(["Average"] + list(averaged_performance))
        csv_lines.append([])

if mode:
    # output results
    result_path = "{result}/{file_name}".format(
        result=param["result_directory"],
        file_name=param["result_file"])
    com.logger.info("AUC and pAUC results -> {}".format(result_path))
    eval_func.save_csv(save_file_path=result_path, save_data=csv_lines)

2020-11-30 18:13:32,777 - INFO - load_directory <- development
2020-11-30 18:13:32,779 - INFO - ===========================
2020-11-30 18:13:32,779 - INFO - [1/6] /media/hiroki/working/research/dcase2020/datasets/DCASE2/dev_data/ToyCar
2020-11-30 18:13:32,780 - INFO - ============== MODEL LOAD ==============
2020-11-30 18:13:33,716 - INFO - ============== CALC GMM PARAM : ToyCar ==============
100%|██████████| 55/55 [00:05<00:00, 10.50it/s]
2020-11-30 18:13:38,959 - INFO - target_dir : /media/hiroki/working/research/dcase2020/datasets/DCASE2/dev_data/ToyCar_id_01
2020-11-30 18:13:38,966 - INFO - test_file  num : 601
2020-11-30 18:13:38,967 - INFO - ============== BEGIN TEST FOR A MACHINE ID ==============
  0%|          | 0/5 [00:00<?, ?it/s]

N: 7000
phi :
 tensor([3.3541e-05, 9.9992e-01, 4.3399e-05])
mu :
 tensor([[ 0.0985, -0.0885,  0.0955, -0.1044, -0.1037,  0.2007],
        [-0.0246,  0.0311, -0.0232,  0.0162,  0.0272,  0.0713],
        [ 0.1417, -0.1353,  0.1362, -0.1435, -0.1439,  0.2440]])
cov :
 tensor([[[ 0.0545, -0.0517,  0.0499, -0.0510, -0.0510,  0.0513],
         [-0.0517,  0.0542, -0.0513,  0.0513,  0.0513, -0.0513],
         [ 0.0499, -0.0513,  0.0548, -0.0503, -0.0523,  0.0514],
         [-0.0510,  0.0513, -0.0503,  0.0553,  0.0513, -0.0514],
         [-0.0510,  0.0513, -0.0523,  0.0513,  0.0549, -0.0514],
         [ 0.0513, -0.0513,  0.0514, -0.0514, -0.0514,  0.0570]],

        [[ 0.0065, -0.0035,  0.0019, -0.0027, -0.0029,  0.0016],
         [-0.0035,  0.0057, -0.0031,  0.0029,  0.0030, -0.0015],
         [ 0.0019, -0.0031,  0.0067, -0.0021, -0.0038,  0.0016],
         [-0.0027,  0.0029, -0.0021,  0.0072,  0.0031, -0.0015],
         [-0.0029,  0.0030, -0.0038,  0.0031,  0.0065, -0.0016],
         [ 0.0016

100%|██████████| 5/5 [00:01<00:00,  4.90it/s]
2020-11-30 18:13:39,990 - INFO - anomaly score result ->  /media/hiroki/working/research/dcase2020/result/2D/DAGMM/strict_comp/latent5_mixture3/result/anomaly_score_ToyCar_id_01.csv
2020-11-30 18:13:39,994 - INFO - AUC : 0.5487877063175868
2020-11-30 18:13:39,994 - INFO - pAUC : 0.5305994068837432
2020-11-30 18:13:39,995 - INFO - ============ END OF TEST FOR A MACHINE ID ============
2020-11-30 18:13:39,996 - INFO - target_dir : /media/hiroki/working/research/dcase2020/datasets/DCASE2/dev_data/ToyCar_id_02
2020-11-30 18:13:40,004 - INFO - test_file  num : 602
2020-11-30 18:13:40,005 - INFO - ============== BEGIN TEST FOR A MACHINE ID ==============
  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00,  5.85it/s]
2020-11-30 18:13:40,864 - INFO - anomaly score result ->  /media/hiroki/working/research/dcase2020/result/2D/DAGMM/strict_comp/latent5_mixture3/result/anomaly_score_ToyCar_id_02.csv
2020-11-30 18:13:40,867 - INFO - AUC : 0.5693310657596372
2020-11-30 18:13:40,867 - INFO - pAUC : 0.5479174125790667
2020-11-30 18:13:40,868 - INFO - ============ END OF TEST FOR A MACHINE ID ============
2020-11-30 18:13:40,868 - INFO - target_dir : /media/hiroki/working/research/dcase2020/datasets/DCASE2/dev_data/ToyCar_id_03
2020-11-30 18:13:40,876 - INFO - test_file  num : 602
2020-11-30 18:13:40,877 - INFO - ============== BEGIN TEST FOR A MACHINE ID ==============
  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00,  6.32it/s]
2020-11-30 18:13:41,671 - INFO - anomaly score result ->  /media/hiroki/working/research/dcase2020/result/2D/DAGMM/strict_comp/latent5_mixture3/result/anomaly_score_ToyCar_id_03.csv
2020-11-30 18:13:41,674 - INFO - AUC : 0.69578231292517
2020-11-30 18:13:41,675 - INFO - pAUC : 0.5898675259577515
2020-11-30 18:13:41,675 - INFO - ============ END OF TEST FOR A MACHINE ID ============
2020-11-30 18:13:41,676 - INFO - target_dir : /media/hiroki/working/research/dcase2020/datasets/DCASE2/dev_data/ToyCar_id_04
2020-11-30 18:13:41,685 - INFO - test_file  num : 602
2020-11-30 18:13:41,685 - INFO - ============== BEGIN TEST FOR A MACHINE ID ==============
  0%|          | 0/5 [00:00<?, ?it/s]

...

N: 5822
phi :
 tensor([0.0056, 0.0063, 0.9881])
mu :
 tensor([[ 0.1357, -0.1275, -0.1340, -0.1349, -0.1271,  0.1523],
        [ 0.1350, -0.1273, -0.1331, -0.1335, -0.1271,  0.1486],
        [-0.0062,  0.0062,  0.0080,  0.0095,  0.0077,  0.0648]])
cov :
 tensor([[[ 0.0661, -0.0619, -0.0661, -0.0670, -0.0629,  0.0356],
         [-0.0619,  0.0582,  0.0617,  0.0627,  0.0588, -0.0335],
         [-0.0661,  0.0617,  0.0666,  0.0673,  0.0631, -0.0356],
         [-0.0670,  0.0627,  0.0673,  0.0684,  0.0635, -0.0363],
         [-0.0629,  0.0588,  0.0631,  0.0635,  0.0603, -0.0341],
         [ 0.0356, -0.0335, -0.0356, -0.0363, -0.0341,  0.0346]],

        [[ 0.0649, -0.0609, -0.0649, -0.0657, -0.0618,  0.0341],
         [-0.0609,  0.0574,  0.0607,  0.0615,  0.0579, -0.0321],
         [-0.0649,  0.0607,  0.0652,  0.0659,  0.0619, -0.0342],
         [-0.0657,  0.0615,  0.0659,  0.0669,  0.0623, -0.0348],
         [-0.0618,  0.0579,  0.0619,  0.0623,  0.0593, -0.0327],
         [ 0.0341, -0.0321, -

100%|██████████| 2/2 [00:00<00:00,  4.70it/s]
2020-11-30 18:14:18,305 - INFO - anomaly score result ->  /media/hiroki/working/research/dcase2020/result/2D/DAGMM/strict_comp/latent5_mixture3/result/anomaly_score_valve_id_00.csv
2020-11-30 18:14:18,308 - INFO - AUC : 0.6691150442477877
2020-11-30 18:14:18,309 - INFO - pAUC : 0.5556590591523055
2020-11-30 18:14:18,310 - INFO - ============ END OF TEST FOR A MACHINE ID ============
2020-11-30 18:14:18,310 - INFO - target_dir : /media/hiroki/working/research/dcase2020/datasets/DCASE2/dev_data/valve_id_02
2020-11-30 18:14:18,315 - INFO - test_file  num : 214
2020-11-30 18:14:18,315 - INFO - ============== BEGIN TEST FOR A MACHINE ID ==============
  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00,  4.81it/s]
2020-11-30 18:14:18,734 - INFO - anomaly score result ->  /media/hiroki/working/research/dcase2020/result/2D/DAGMM/strict_comp/latent5_mixture3/result/anomaly_score_valve_id_02.csv
2020-11-30 18:14:18,737 - INFO - AUC : 0.45026315789473687
2020-11-30 18:14:18,737 - INFO - pAUC : 0.49584487534626037
2020-11-30 18:14:18,738 - INFO - ============ END OF TEST FOR A MACHINE ID ============
2020-11-30 18:14:18,739 - INFO - target_dir : /media/hiroki/working/research/dcase2020/datasets/DCASE2/dev_data/valve_id_04
2020-11-30 18:14:18,743 - INFO - test_file  num : 214
2020-11-30 18:14:18,743 - INFO - ============== BEGIN TEST FOR A MACHINE ID ==============
  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00,  4.67it/s]
2020-11-30 18:14:19,175 - INFO - anomaly score result ->  /media/hiroki/working/research/dcase2020/result/2D/DAGMM/strict_comp/latent5_mixture3/result/anomaly_score_valve_id_04.csv
2020-11-30 18:14:19,178 - INFO - AUC : 0.787719298245614
2020-11-30 18:14:19,178 - INFO - pAUC : 0.5304709141274239
2020-11-30 18:14:19,179 - INFO - ============ END OF TEST FOR A MACHINE ID ============
2020-11-30 18:14:19,179 - INFO - target_dir : /media/hiroki/working/research/dcase2020/datasets/DCASE2/dev_data/valve_id_06
2020-11-30 18:14:19,183 - INFO - test_file  num : 214
2020-11-30 18:14:19,184 - INFO - ============== BEGIN TEST FOR A MACHINE ID ==============
  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00,  4.57it/s]
2020-11-30 18:14:19,625 - INFO - anomaly score result ->  /media/hiroki/working/research/dcase2020/result/2D/DAGMM/strict_comp/latent5_mixture3/result/anomaly_score_valve_id_06.csv
2020-11-30 18:14:19,629 - INFO - AUC : 0.6468421052631579
2020-11-30 18:14:19,630 - INFO - pAUC : 0.5124653739612188
2020-11-30 18:14:19,630 - INFO - ============ END OF TEST FOR A MACHINE ID ============
2020-11-30 18:14:19,631 - INFO - AUC and pAUC results -> /media/hiroki/working/research/dcase2020/result/2D/DAGMM/strict_comp/latent5_mixture3/result/result.csv
